In [1]:
import gensim
import numpy as np
import string


from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import Placeholders




pretrained_word_vector_binary = '/Users/saur6410/Google Drive/VT/Old/Thesis/Source/ThesisPython/data/GoogleNews-vectors-negative300.bin'






class word2vec:
    model = None
    models = {}
    stop = set(stopwords.words('english'))
    wordnet_lemmatizer = WordNetLemmatizer()

    def get_model(self):
        if not self.model:
            print("Loading word2vec model from file:{}".format(pretrained_word_vector_binary))
            model = gensim.models.KeyedVectors.load_word2vec_format(pretrained_word_vector_binary, binary=True)
            model.init_sims(replace=True)  # we are no longer training the model so allow it to trim memory
            word2vec.model = model

        return self.model


    def get_model_from_file(self,name):
        if not name in word2vec.models:
            sentences = []
            with open(name,"r") as f:
                sentences = map(lambda x: word2vec.extract_sentence(x).split(), f.readlines())
            file_model = gensim.models.Word2Vec(sentences=sentences, size=300, min_count=1, window=5)
            # sentences = []
            # sentences.append('ebola threat real allow african conference nyc risky stupid wrong'.split())
            # file_model.similar_by_vector(sentences)
            word2vec.models[name] = file_model.wv
        return word2vec.models[name]


    def extract_sentence(self, line):
        lineContent = line.split(',')
        return (lineContent[1] if len(lineContent) > 1 else lineContent[0])

    def avg_feature_vector(self, words, model, num_features , index2word_set):
        # function to average all words vectors in a given paragraph
        featureVec = np.zeros((num_features,), dtype="float32")
        nwords = 0
        model = self.get_model()

        # list containing names of words in the vocabulary
        # index2word_set = set(model.index2word) this is moved as input param for performance reasons
        for word in words:
            if word in index2word_set:
                nwords += 1
                featureVec = np.add(featureVec, model[word])

        if (nwords > 0):
            featureVec = np.divide(featureVec, nwords)
        return featureVec

    def get_word_vector(self, word):
        model = self.get_model()
        vec = model[word]
        return vec[0:Placeholders.n_inputs]

    def get_sentence_vector(self, sentence):
        featureVec = np.zeros((1,300))
        nwords = 0
        clean_sentence = self.clean_sent(self.wordnet_lemmatizer, sentence)
        model = self.get_model()
        words = clean_sentence.rstrip().split(" ")
        if(len(words) == 0):
            return featureVec
        for word in words:
            try:
                word_vector = model[word]
                nwords += 1
                #print word_vector
                featureVec = np.add(featureVec, word_vector)
            except:
                pass #Swallow exception
        if (nwords > 0):
            featureVec = np.divide(featureVec, nwords)
        return featureVec

    def get_sentence_matrix(self, sentence):
        sentence_matrix = []
        featureVec = np.zeros((1,300))
        nwords = 0
        clean_sentence = self.clean_sent(self.wordnet_lemmatizer, sentence)
        model = self.get_model()
        words = clean_sentence.rstrip().split(" ")
        if(len(words) == 0):
            return sentence_matrix
        for word in words:
            try:
                word_vector = self.get_word_vector(word)
                nwords += 1
                #print word_vector
                sentence_matrix.append(word_vector)
            except:
                pass #Swallow exception
        padded_sentence_matrix = self.getPaddedSentenceMatrix(np.array(sentence_matrix))
        return padded_sentence_matrix

    def get_sentence_vector_ex(self, sentence):
        try:
            #return np.zeros((1, Placeholders.n_steps * Placeholders.n_inputs))
            sentence_matrix = self.get_sentence_matrix(sentence)
            return sentence_matrix.reshape((1, Placeholders.n_steps * Placeholders.n_inputs))
        except:
            print("Error with sentence:" + sentence)
        return np.zeros((1, Placeholders.n_steps* Placeholders.n_inputs))

    def getPaddedSentenceMatrix(self, sentenceMatrix):
        wordCount = Placeholders.n_steps
        #print(sentenceMatrix.shape)
        return np.vstack((sentenceMatrix,
                        np.zeros((wordCount - np.shape(sentenceMatrix)[0], np.shape(sentenceMatrix)[1]),
                        dtype=np.float32)))

    def read_from_file(self, name, model):
        with open(name,"r") as f:
            lines = f.readlines()
            tweets_only = map(lambda line: line.split(',')[1].rstrip().split(" "), lines)
            lables_only = map(lambda line: line.split(',')[0].rstrip(), lines)
            vecs = map(lambda t: self.avg_feature_vector(t, model, 300, model.index2word),tweets_only)
            return zip(lables_only,vecs)

    def full_pipeline(self, lem, word):
        word = word.lower()
        word = word.translate(string.punctuation)
        for val in ['a', 'v', 'n']:
            word = lem.lemmatize(word, pos=val)
        return word


    def clean_sent(self, lem, sent):
        #sent = unicode(sent,errors='ignore')
        words = sent.replace(","," ").replace(";", " ").replace("#"," ").replace(":", " ").replace("@", " ").split()
        filtered_words = filter(lambda word: word.isalpha() and len(word) > 1 and word != "http" and word != "rt", [self.full_pipeline(lem, word) for word in words])
        return ' '.join(self.filter_stopwords(filtered_words))

    def filter_stopwords(self, words):
        return filter(lambda word: word not in self.stop, words)


# sentence = "Gala Bingo clubs bought for 241m: The UK's largest High Street bingo operator, Gala, is being taken over by_ https://t.co/HzeeykJUd3"
# wv = word2vec()
# clean_sentence = wv.clean_sent(wv.wordnet_lemmatizer,sentence)
# print(clean_sentence)
# sentence_matrix = wv.get_sentence_matrix(clean_sentence)
# model = word2vec().get_model()

if __name__ == "__main__":
    sentence = "Gala Bingo clubs bought for 241m: The UK's largest High Street bingo operator, Gala, is being taken over by_ https://t.co/HzeeykJUd3"
    wv = word2vec()
    clean_sentence = wv.clean_sent(wv.wordnet_lemmatizer,sentence)
    print(clean_sentence)
    sentence_matrix = wv.get_sentence_matrix(clean_sentence)

    model = word2vec().get_model()
    # you can find the terms that are similar to a list of words and different from
    # another list of words like so
    print(model.most_similar(positive=['hurricane'], negative=['isaac']))

    # you can also get the vector for a specific word by doing
    print(model['hurricane'])

    # you can ask for similarity by doing
    print(model.similarity('hurricane', 'shooting'))

    print("done")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


gala bingo club buy large high street bingo operator gala take
Loading word2vec model from file:/Users/saur6410/Google Drive/VT/Old/Thesis/Source/ThesisPython/data/GoogleNews-vectors-negative300.bin
[('hurricanes', 0.5880494117736816), ('Hurricane_Ike', 0.5491310358047485), ('Hurricane_Rita', 0.5269111394882202), ('Hurricane_Gustav', 0.5230242013931274), ('Hurricane_Katrina', 0.522605299949646), ('Hurricane_Wilma', 0.5225527286529541), ('Hurricane_Ivan', 0.5218238830566406), ('Hurricane_Frances', 0.5146640539169312), ('Hurricane_Charley', 0.5122226476669312), ('storm', 0.4903242886066437)]
[  3.66145410e-02  -2.80793943e-02  -9.30207297e-02   2.64713243e-02
   1.62044093e-02   4.40364070e-02  -4.25520353e-02  -7.17447102e-02
   1.41510248e-01  -4.60774219e-03   4.79947366e-02  -5.78905605e-02
   4.45312001e-02  -2.00390406e-02   7.17447102e-02   6.87759668e-02
   2.16471106e-02   6.40136003e-03  -4.51496895e-03  -8.75780284e-02
   8.41144845e-02  -5.22004627e-02   6.97655454e-02   3.43